In [9]:
import csv
from azure.storage.blob import BlobServiceClient
import snowflake.connector

In [10]:
# Azure Storage container details
azure_storage_connection_string = 'DefaultEndpointsProtocol=https;AccountName=pppdatastorage;AccountKey=b4UhnaSHwcv1Tm7MNL0lYCaC6BLMh2T1A5EjU4fgHxltuPg2jCeIPZXgjcIDohKHjtKQe9Qegbso+ASto63eZA==;EndpointSuffix=core.windows.net'
azure_storage_container = 'pppdata'

In [11]:
# Snowflake connection details
snowflake_account = ''
snowflake_user = ''
snowflake_password = ''
snowflake_database = 'PPP_LOAN_DB'
snowflake_schema = 'PPP_STAGING_DATA'
snowflake_warehouse= 'The_PPP_Loan_Pro_WH'
snowflake_table='LoanFact'


In [12]:
# Connect to Azure Storage
blob_service_client = BlobServiceClient.from_connection_string(azure_storage_connection_string)
container_client = blob_service_client.get_container_client(azure_storage_container)


In [13]:
# Download the CSV file from Azure Storage
csv_file_name = 'CompleteLoan_Fact'

blob_client = container_client.get_blob_client(csv_file_name)
downloaded_blob = blob_client.download_blob()
csv_data = downloaded_blob.content_as_text()

# Convert CSV data to a list of dictionaries
csv_data_lines = csv_data.split("\n")
csv_reader = csv.DictReader(csv_data_lines)
data = [dict(row) for row in csv_reader]
#print(data)

In [14]:
# Connect to Snowflake
conn = snowflake.connector.connect(
    user=snowflake_user,
    password=snowflake_password,
    account=snowflake_account,
    database=snowflake_database,
    schema=snowflake_schema,
    warehouse=snowflake_warehouse
)

In [15]:
# Create a cursor to execute SQL statements
cursor = conn.cursor()

# Set the current schema
set_schema_sql = f"USE SCHEMA {snowflake_schema}"
cursor.execute(set_schema_sql)

# Create a warehouse in Snowflake
create_warehouse_sql = f'''
    CREATE WAREHOUSE IF NOT EXISTS {snowflake_warehouse}
    WITH WAREHOUSE_SIZE = 'X-Small'
    AUTO_SUSPEND = 600
    AUTO_RESUME = TRUE
    COMMENT = 'The PPP Loan WH'
'''
cursor.execute(create_warehouse_sql)

In [16]:
# Commit the transaction
conn.commit()

# Create table in Snowflake
create_table_sql = f"CREATE TABLE {snowflake_table}(LoanID int, ForgivenessAmount float, InitialApprovalAmount float, CurrentApprovalAmount float, UndisbursedAmount float, LoanNumber int, SBAGuarantyPercentage float, ForgivenessDate VARCHAR,Forgivness_year VARCHAR,Forgiveness_month int,Forgiveness_day int,Forgiveness_quarter int,ForgivenessPK VARCHAR)"
cursor.execute(create_table_sql)

In [ ]:
# Insert CSV data from Azure into Snowflake table
insert_sql = f"INSERT INTO {snowflake_table} VALUES (%s, %s, %s,%s, %s, %s,%s, %s,%s,%s,%s,%s,%s)"
for row in data:
    cursor.execute(insert_sql, (
    row['LoanID'],
    row['ForgivenessAmount'] if row['ForgivenessAmount'] != '' else None,
    row['InitialApprovalAmount'] if row['InitialApprovalAmount'] != '' else None,
    row['CurrentApprovalAmount'] if row['CurrentApprovalAmount'] != '' else None,
    row['UndisbursedAmount'] if row['UndisbursedAmount'] != '' else None,
    row['LoanNumber'],
    row['SBAGuarantyPercentage'] if row['SBAGuarantyPercentage'] != '' else None,
    row['ForgivenessDate'],
    row['Forgivness_year'] if row['Forgivness_year'] != '' else None,
    row['Forgiveness_month'] if row['Forgiveness_month'] != '' else None,
    row['Forgiveness_day'] if row['Forgiveness_day'] != '' else None,
    row['Forgiveness_quarter'] if row['Forgiveness_quarter'] != '' else None,
    row['ForgivenessPK']
))

In [ ]:
# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()